To showcase the cancellation, run the following cell and then send an interrupt to the kernel before it finishes execution. In the Jupyter Notebook interface, that is double tapping `I`, or using the stop button in the top left menu.

In [1]:
from biodivine_aeon import *

bn = BooleanNetwork.from_file("002.aeon")
bn = bn.infer_valid_graph()

fixed_points = FixedPointsComp.create_from(bn).symbolic()

print(
    f"{fixed_points.cardinality()} ({fixed_points.vertices().cardinality()} | {fixed_points.colors().cardinality()})"
)

32768 (32768 | 32)


Notice that there is no usage of a Config class, `FixedPointsComp` can be created straight from the Boolean network.

Next, let's limit the execution of a long computaion to 10 seconds. Here, the Config is created via a Python constructor, but it can also be created with the `create_from()` method, just like `FixedPointsComp` above.
Additionally, sending the interrupt manually still works.

In [2]:
bn = BooleanNetwork.from_file("001.aeon")
bn = bn.infer_valid_graph()

try:
    fixed_points = FixedPointsComp.with_config(FixedPointsConfig(bn, time_limit_millis=10_000)).symbolic()
    print(
        f"{fixed_points.cardinality()} ({fixed_points.vertices().cardinality()} | {fixed_points.colors().cardinality()})"
    )
except Exception as e:
    print("[Error]", e)

[Error] Cancelled: partial_result=1933312


Lastly, let's add some logging. Notice that the logs are printed out continously as the cell executes.

In [3]:
import logging

logging.basicConfig(level=logging.INFO)

bn = BooleanNetwork.from_file("031.aeon")
bn = bn.infer_valid_graph()

ctx = SymbolicSpaceContext(bn)
graph = AsynchronousGraph(bn, ctx)
percolation = PercolationComp.create_from(graph)

space_dict_len_sum, p_space_len_sum, limit = 0, 0, 500
for space in ctx.mk_unit_spaces():
    space_dict = {k: v for k, v in space.items() if v is not None}
    p_space = percolation.percolate_subspace(space_dict)

    space_dict_len_sum += len(space_dict)
    p_space_len_sum += len(p_space)

    limit -= 1
    if limit == 0:
        break

print(f"{space_dict_len_sum} -> {p_space_len_sum}")


INFO:Percolation.percolate_subspace:Started with 9 variables in the subspace.
INFO:Percolation.percolate_subspace:Done. Result: 9 fixed variables.
INFO:Percolation.percolate_subspace:Started with 9 variables in the subspace.
INFO:Percolation.percolate_subspace:Done. Result: 9 fixed variables.
INFO:Percolation.percolate_subspace:Started with 8 variables in the subspace.
INFO:Percolation.percolate_subspace:Done. Result: 9 fixed variables.
INFO:Percolation.percolate_subspace:Started with 9 variables in the subspace.
INFO:Percolation.percolate_subspace:Done. Result: 9 fixed variables.
INFO:Percolation.percolate_subspace:Started with 8 variables in the subspace.
INFO:Percolation.percolate_subspace:Done. Result: 9 fixed variables.
INFO:Percolation.percolate_subspace:Started with 9 variables in the subspace.
INFO:Percolation.percolate_subspace:Done. Result: 9 fixed variables.
INFO:Percolation.percolate_subspace:Started with 8 variables in the subspace.
INFO:Percolation.percolate_subspace:Done

3625 -> 4494
